In [1]:
# LSTM for international airline passengers problem with memory
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [],[] 
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back):]
        dataX.append(a)
        dataY.append(dataset[i:(i + look_back):])
    
    dataX=np.squeeze(dataX,axis=1) #차원 축소
    dataY=np.squeeze(dataY,axis=1) #차원 축소
    return np.array(dataX), np.array(dataY)

In [3]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [4]:
data1=pd.read_excel('x_data.xls') #x좌표 엑셀 데이터 불러옴 
data2=pd.read_excel('y_data.xls') #y좌표 엑셀 데이터 불러옴 
data3=pd.read_excel('speed.xls') #speed좌표 엑셀 데이터 불러옴 

pddata1=pd.DataFrame(data1) 
pddata1.head()
pddata2=pd.DataFrame(data2) 
pddata2.head()
pddata3=pd.DataFrame(data3) 
pddata3.head()

hap=[] #진짜 합
for j in range(len(pddata1)):
    x_data=np.array(pddata1.loc[j])
    y_data=np.array(pddata2.loc[j])
    s_data=np.array(pddata3.loc[j])

    hap1=[] #[x,y,속력] 데이터를 저장하고 있는 리스트
    
    for i in range(len(s_data)): 
        sum=[] #리스트 하나당 임시로 x,y,속력을 저장할 리스트 (for문 돌릴때마다 초기화)
        sum.append(x_data[i]) #x넣음
        sum.append(y_data[i]) #y넣음
        sum.append(s_data[i]) #속력넣음
        hap1.append(sum) #[x,y,속력] 하나의 리스트를 hap리스트에 넣음  

    j = j + 1
    
    hap.append(hap1)

n1 = np.array(hap)
print(n1.shape) #257,99,3 257개의 Sample과 99개 시계열, 3개 feature(x,y,속력)

(257, 99, 3)


In [5]:
num_sample = n1.shape[0] # 257개 Sample 데이터
num_sequence = n1.shape[1] # 99개 시계열 데이터
num_feature = n1.shape[2] #3개 Feature

dataset = n1

In [6]:
dataset

array([[[388.        , 252.        ,   0.        ],
        [388.        , 252.        ,   0.        ],
        [388.        , 252.        ,   0.        ],
        ...,
        [224.        , 357.        ,   0.        ],
        [224.        , 357.        ,   0.        ],
        [224.        , 357.        ,   0.        ]],

       [[210.        , 371.        ,   0.        ],
        [210.        , 371.        ,   0.        ],
        [210.        , 371.        ,   0.        ],
        ...,
        [335.        , 426.        ,   0.        ],
        [335.        , 426.        ,   0.        ],
        [335.        , 426.        ,   2.        ]],

       [[563.        , 328.        ,   0.        ],
        [563.        , 328.        ,   0.        ],
        [563.        , 328.        ,   0.        ],
        ...,
        [407.        , 433.        ,   0.        ],
        [407.        , 433.        ,   0.        ],
        [407.        , 433.        ,   0.        ]],

       ...,

      

In [8]:
train_size = int(len(dataset) * 0.70) #학습 데이터 70%
test_size = len(dataset) - train_size #테스트 데이터 30%
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [9]:
train

array([[[388.        , 252.        ,   0.        ],
        [388.        , 252.        ,   0.        ],
        [388.        , 252.        ,   0.        ],
        ...,
        [224.        , 357.        ,   0.        ],
        [224.        , 357.        ,   0.        ],
        [224.        , 357.        ,   0.        ]],

       [[210.        , 371.        ,   0.        ],
        [210.        , 371.        ,   0.        ],
        [210.        , 371.        ,   0.        ],
        ...,
        [335.        , 426.        ,   0.        ],
        [335.        , 426.        ,   0.        ],
        [335.        , 426.        ,   2.        ]],

       [[563.        , 328.        ,   0.        ],
        [563.        , 328.        ,   0.        ],
        [563.        , 328.        ,   0.        ],
        ...,
        [407.        , 433.        ,   0.        ],
        [407.        , 433.        ,   0.        ],
        [407.        , 433.        ,   0.        ]],

       ...,

      

In [10]:
test

array([[[383.        , 461.        ,   0.        ],
        [383.        , 461.        ,   0.        ],
        [383.        , 461.        ,   0.        ],
        ...,
        [414.        , 396.        ,   0.        ],
        [414.        , 396.        ,   4.24264069],
        [417.        , 393.        ,   0.        ]],

       [[511.        , 314.        ,   0.        ],
        [511.        , 314.        ,   0.        ],
        [511.        , 314.        ,   0.        ],
        ...,
        [531.        , 443.        ,   0.        ],
        [531.        , 443.        ,   1.41421356],
        [532.        , 444.        ,   0.        ]],

       [[534.        , 456.        ,   0.        ],
        [534.        , 456.        ,   0.        ],
        [534.        , 456.        ,   0.        ],
        ...,
        [424.        , 532.        ,   0.        ],
        [424.        , 532.        ,   0.        ],
        [424.        , 532.        ,   2.82842712]],

       ...,

      

In [11]:
np.shape(dataset)

(257, 99, 3)

In [12]:
np.shape(train)

(179, 99, 3)

In [13]:
np.shape(test)

(78, 99, 3)

In [14]:
look_back = 1 #이전 시간 단계 입력변수
#X는 지금 t 값이고, Y는 그 다음의 t+1 값임 (즉, X=t, Y=t+1)
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [15]:
np.shape(trainX)

(177, 99, 3)

In [16]:
np.shape(testX)

(76, 99, 3)

In [17]:
np.shape(trainY)

(177, 99, 3)

In [18]:
np.shape(testY)

(76, 99, 3)

In [19]:
from keras.layers import TimeDistributed
from keras.layers import InputLayer
from keras.layers import Reshape
from keras.layers import *
from keras.models import *
from keras.utils import *

model = Sequential() # Sequeatial Model 

model.add(LSTM(32, activation='relu',return_sequences=True, input_shape=(num_sequence,num_feature)))  
# 20: 메모리 셀의 개수 input_shape: (timestep, feature) 
model.add(Dense(3, activation='softmax')) # 3개의 예측

model.compile(loss='mean_squared_error', optimizer='adam') 

model.fit(trainX, trainY, epochs=150,batch_size=10, validation_split=0.2, verbose=1)

model.summary()

Train on 141 samples, validate on 36 samples
Epoch 1/150
141/141 [==============================] - 3s 18ms/step - loss: 117131.0363 - val_loss: 112812.8602
Epoch 2/150
141/141 [==============================] - 1s 8ms/step - loss: 117131.7835 - val_loss: 112816.2769
Epoch 3/150
141/141 [==============================] - 1s 10ms/step - loss: 117132.8364 - val_loss: 112812.9631
Epoch 4/150
141/141 [==============================] - 1s 9ms/step - loss: 117116.6319 - val_loss: 112792.4353
Epoch 5/150
141/141 [==============================] - 1s 9ms/step - loss: 117109.2666 - val_loss: 112791.1732
Epoch 6/150
141/141 [==============================] - 1s 10ms/step - loss: 117108.8338 - val_loss: 112792.4197
Epoch 7/150
141/141 [==============================] - 1s 8ms/step - loss: 117108.3018 - val_loss: 112791.6623
Epoch 8/150
141/141 [==============================] - 1s 10ms/step - loss: 117107.9881 - val_loss: 112790.2214
Epoch 9/150
141/141 [==============================] - 1s 10ms/

Epoch 74/150
141/141 [==============================] - 1s 9ms/step - loss: 117105.6329 - val_loss: 112789.5195
Epoch 75/150
141/141 [==============================] - 1s 7ms/step - loss: 117105.6058 - val_loss: 112789.4944
Epoch 76/150
141/141 [==============================] - 1s 7ms/step - loss: 117105.5870 - val_loss: 112789.4692
Epoch 77/150
141/141 [==============================] - 1s 9ms/step - loss: 117105.6417 - val_loss: 112789.4440
Epoch 78/150
141/141 [==============================] - 1s 8ms/step - loss: 117105.5999 - val_loss: 112789.4145
Epoch 79/150
141/141 [==============================] - 1s 8ms/step - loss: 117105.5330 - val_loss: 112789.3893
Epoch 80/150
141/141 [==============================] - 1s 7ms/step - loss: 117105.5624 - val_loss: 112788.9627
Epoch 81/150
141/141 [==============================] - 1s 7ms/step - loss: 117105.5177 - val_loss: 112788.8941
Epoch 82/150
141/141 [==============================] - 1s 8ms/step - loss: 117105.5030 - val_loss: 1127

141/141 [==============================] - 1s 8ms/step - loss: 117103.3088 - val_loss: 112787.5078
Epoch 147/150
141/141 [==============================] - 1s 7ms/step - loss: 117103.3103 - val_loss: 112787.5078
Epoch 148/150
141/141 [==============================] - 1s 8ms/step - loss: 117103.3043 - val_loss: 112787.5078
Epoch 149/150
141/141 [==============================] - 1s 8ms/step - loss: 117103.3025 - val_loss: 112787.5078
Epoch 150/150
141/141 [==============================] - 1s 8ms/step - loss: 117103.3136 - val_loss: 112787.5078
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 99, 32)            4608      
_________________________________________________________________
dense_1 (Dense)              (None, 99, 3)             99        
Total params: 4,707
Trainable params: 4,707
Non-trainable params: 0
_____________________________

In [20]:
print("정확도: ",(model.evaluate(testX, testY)))

trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
print('y: ',testY,', predict: ',model.predict(trainX).flatten())

76/76 [==============================] - 0s 1ms/step
정확도:  118583.39761513157
y:  [[[383.         461.           0.        ]
  [383.         461.           0.        ]
  [383.         461.           0.        ]
  ...
  [414.         396.           0.        ]
  [414.         396.           4.24264069]
  [417.         393.           0.        ]]

 [[511.         314.           0.        ]
  [511.         314.           0.        ]
  [511.         314.           0.        ]
  ...
  [531.         443.           0.        ]
  [531.         443.           1.41421356]
  [532.         444.           0.        ]]

 [[534.         456.           0.        ]
  [534.         456.           0.        ]
  [534.         456.           0.        ]
  ...
  [424.         532.           0.        ]
  [424.         532.           0.        ]
  [424.         532.           2.82842712]]

 ...

 [[514.         637.           0.        ]
  [514.         637.           0.        ]
  [514.         637.        